## Assignment two: Named entity recognition

##### Necessary imports

In [13]:
import pandas as pd 
import numpy as np 
import re
import nltk
nltk.download('punkt')  # Download tokenizer for Arabic
from nltk.tokenize import word_tokenize
import sklearn 
import sklearn_crfsuite
from sklearn_crfsuite import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Data creation

In [2]:
data = [
    {"sentence": "زار محمد دبي في ٢٠٢٣.", "entities": {"Person": "محمد", "Location": "دبي", "Date": "٢٠٢٣"}},
    {"sentence": "حضرت مريم مؤتمر جوجل في باريس.", "entities": {"Person": "مريم", "Organization": "جوجل", "Location": "باريس"}},
    {"sentence": "أسس أحمد شركة الفجر في عام ٢٠١٩.", "entities": {"Person": "أحمد", "Organization": "شركة الفجر", "Date": "٢٠١٩"}},
    {"sentence": "تعمل سارة في مستشفى الجامعة في بيروت.", "entities": {"Person": "سارة", "Organization": "مستشفى الجامعة", "Location": "بيروت"}},
    {"sentence": "أقيمت الدورة الأولمبية في طوكيو ٢٠٢١.", "entities": {"Event": "الدورة الأولمبية", "Location": "طوكيو", "Date": "٢٠٢١"}},
    {"sentence": "قدم خالد مشروعه في اجتماع مايكروسوفت.", "entities": {"Person": "خالد", "Organization": "مايكروسوفت", "Event": "اجتماع"}},
    {"sentence": "افتتح وليد مطعم الورد في جدة.", "entities": {"Person": "وليد", "Organization": "مطعم الورد", "Location": "جدة"}},
    {"sentence": "التقى علي بشركة آبل في مؤتمر دبي.", "entities": {"Person": "علي", "Organization": "آبل", "Event": "مؤتمر", "Location": "دبي"}},
    {"sentence": "أرسلت ندى طلبًا إلى جامعة القاهرة.", "entities": {"Person": "ندى", "Organization": "جامعة القاهرة", "Location": "القاهرة"}},
    {"sentence": "اشترى يوسف سيارة من تويوتا في ٢٠٢٠.", "entities": {"Person": "يوسف", "Organization": "تويوتا", "Date": "٢٠٢٠"}},
    {"sentence": "سافرت ليلى إلى الإسكندرية في الصيف.", "entities": {"Person": "ليلى", "Location": "الإسكندرية", "Date": "الصيف"}},
    {"sentence": "عقدت سامسونج مؤتمرًا في الرياض.", "entities": {"Organization": "سامسونج", "Event": "مؤتمر", "Location": "الرياض"}},
    {"sentence": "يعيش طارق في القاهرة منذ عام ٢٠١٨.", "entities": {"Person": "طارق", "Location": "القاهرة", "Date": "٢٠١٨"}},
    {"sentence": "شاركت نور في سباق ماراثون بيروت.", "entities": {"Person": "نور", "Event": "سباق ماراثون", "Location": "بيروت"}},
    {"sentence": "قام الرئيس بزيارة المملكة العربية السعودية في مارس.", "entities": {"Person": "الرئيس", "Location": "المملكة العربية السعودية", "Date": "مارس"}},
    {"sentence": "أطلقت فيسبوك ميزة جديدة في ٢٠٢٢.", "entities": {"Organization": "فيسبوك", "Date": "٢٠٢٢"}},
    {"sentence": "شارك عمر في اجتماع هيئة الأمم المتحدة.", "entities": {"Person": "عمر", "Organization": "هيئة الأمم المتحدة", "Event": "اجتماع"}},
    {"sentence": "فازت مصر ببطولة كأس العالم في ٢٠٢٢.", "entities": {"Location": "مصر", "Event": "بطولة كأس العالم", "Date": "٢٠٢٢"}},
    {"sentence": "زار ماجد متحف اللوفر في باريس.", "entities": {"Person": "ماجد", "Location": "متحف اللوفر", "Location": "باريس"}},
    {"sentence": "ألقى عبد الله خطابًا في منتدى الأمم المتحدة.", "entities": {"Person": "عبد الله", "Event": "خطاب", "Organization": "منتدى الأمم المتحدة"}},
    {"sentence": "احتفلت عائلة حسان بعيد ميلاد لينا في لبنان.", "entities": {"Person": "لينا", "Location": "لبنان"}},
    {"sentence": "زار وفد من الصين مقر الأمم المتحدة في نيويورك.", "entities": {"Location": "الصين", "Organization": "الأمم المتحدة", "Location": "نيويورك"}},
    {"sentence": "يعمل علي في شركة مايكروسوفت منذ ٢٠١٥.", "entities": {"Person": "علي", "Organization": "شركة مايكروسوفت", "Date": "٢٠١٥"}},
    {"sentence": "أعلنت أمازون عن افتتاح مقر جديد في لندن.", "entities": {"Organization": "أمازون", "Location": "لندن"}},
    {"sentence": "زارت ليلى متحف القاهرة في ٢٠٢١.", "entities": {"Person": "ليلى", "Location": "متحف القاهرة", "Date": "٢٠٢١"}},
    {"sentence": "أطلق سامر كتابًا جديدًا في معرض الكتاب في الدوحة.", "entities": {"Person": "سامر", "Event": "معرض الكتاب", "Location": "الدوحة"}},
    {"sentence": "أسس مصطفى مدرسة الإبداع في الإمارات.", "entities": {"Person": "مصطفى", "Organization": "مدرسة الإبداع", "Location": "الإمارات"}},
    {"sentence": "ألقى الرئيس خطابًا في مؤتمر القمة العربية.", "entities": {"Person": "الرئيس", "Event": "مؤتمر القمة العربية"}},
    {"sentence": "قدمت رنا تقريرًا في ندوة جامعة الملك فهد.", "entities": {"Person": "رنا", "Event": "ندوة", "Organization": "جامعة الملك فهد"}},
    {"sentence": "فاز فريق الهلال بكأس آسيا في ٢٠٢٠.", "entities": {"Organization": "فريق الهلال", "Event": "كأس آسيا", "Date": "٢٠٢٠"}},
    {"sentence": "اشترت فاطمة منتجات من سوق أمازون في ٢٠٢٢.", "entities": {"Person": "فاطمة", "Organization": "أمازون", "Date": "٢٠٢٢"}},
    {"sentence": "ألقى نادر محاضرة في جامعة بيروت العربية.", "entities": {"Person": "نادر", "Organization": "جامعة بيروت العربية", "Event": "محاضرة"}},
    {"sentence": "عقدت شركة جوجل ندوة في القاهرة.", "entities": {"Organization": "شركة جوجل", "Event": "ندوة", "Location": "القاهرة"}},
    {"sentence": "التقت نورهان بالسفيرة في اجتماع الأمم المتحدة.", "entities": {"Person": "نورهان", "Person": "السفيرة", "Event": "اجتماع", "Organization": "الأمم المتحدة"}},
    {"sentence": "سافرت منى إلى تركيا لحضور مؤتمر.", "entities": {"Person": "منى", "Location": "تركيا", "Event": "مؤتمر"}},
    {"sentence": "زار عمر برج خليفة في دبي.", "entities": {"Person": "عمر", "Location": "برج خليفة", "Location": "دبي"}},
    {"sentence": "أعلنت شركة سامسونج عن هاتف جديد في ٢٠٢٣.", "entities": {"Organization": "شركة سامسونج", "Date": "٢٠٢٣"}},
    {"sentence": "فاز منتخب السعودية ببطولة الخليج في ٢٠٢١.", "entities": {"Organization": "منتخب السعودية", "Event": "بطولة الخليج", "Date": "٢٠٢١"}},
    {"sentence": "حضر خالد معرض الكتاب في الشارقة.", "entities": {"Person": "خالد", "Event": "معرض الكتاب", "Location": "الشارقة"}},
    {"sentence": "سافرت ياسمين إلى اليابان لقضاء العطلة الصيفية.", "entities": {"Person": "ياسمين", "Location": "اليابان", "Date": "العطلة الصيفية"}}
]


##### Initial inspection

In [3]:
df = pd.DataFrame(data)
df.head()

,sentence,entities
0,زار محمد دبي في ٢٠٢٣.,"{'Person': 'محمد', 'Location': 'دبي', 'Date': ..."
1,حضرت مريم مؤتمر جوجل في باريس.,"{'Person': 'مريم', 'Organization': 'جوجل', 'Lo..."
2,أسس أحمد شركة الفجر في عام ٢٠١٩.,"{'Person': 'أحمد', 'Organization': 'شركة الفجر..."
3,تعمل سارة في مستشفى الجامعة في بيروت.,"{'Person': 'سارة', 'Organization': 'مستشفى الج..."
4,أقيمت الدورة الأولمبية في طوكيو ٢٠٢١.,"{'Event': 'الدورة الأولمبية', 'Location': 'طوك..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  40 non-null     object
 1   entities  40 non-null     object
dtypes: object(2)
memory usage: 772.0+ bytes


##### Cleaning functions

In [8]:
def preprocess_text(text):
    text = re.sub("[إأآا]", "ا", text)  
    text = re.sub("ى", "ي", text)       
    text = re.sub("ؤ", "ء", text)       
    text = re.sub("ئ", "ء", text)
    
    text = re.sub(r'[\u064B-\u065F]', '', text)
    
    text = re.sub(r'[^\w\s]', '', text)
    
    return text



In [9]:
def tokenize_text(text):
    tokens = word_tokenize(text)  
    return tokens



In [10]:
df['cleaned_sentence'] = df['sentence'].apply(preprocess_text)
df['tokens'] = df['cleaned_sentence'].apply(tokenize_text)

df.head()


,sentence,entities,cleaned_sentence,tokens
0,زار محمد دبي في ٢٠٢٣.,"{'Person': 'محمد', 'Location': 'دبي', 'Date': ...",زار محمد دبي في ٢٠٢٣,"[زار, محمد, دبي, في, ٢٠٢٣]"
1,حضرت مريم مؤتمر جوجل في باريس.,"{'Person': 'مريم', 'Organization': 'جوجل', 'Lo...",حضرت مريم مءتمر جوجل في باريس,"[حضرت, مريم, مءتمر, جوجل, في, باريس]"
2,أسس أحمد شركة الفجر في عام ٢٠١٩.,"{'Person': 'أحمد', 'Organization': 'شركة الفجر...",اسس احمد شركة الفجر في عام ٢٠١٩,"[اسس, احمد, شركة, الفجر, في, عام, ٢٠١٩]"
3,تعمل سارة في مستشفى الجامعة في بيروت.,"{'Person': 'سارة', 'Organization': 'مستشفى الج...",تعمل سارة في مستشفي الجامعة في بيروت,"[تعمل, سارة, في, مستشفي, الجامعة, في, بيروت]"
4,أقيمت الدورة الأولمبية في طوكيو ٢٠٢١.,"{'Event': 'الدورة الأولمبية', 'Location': 'طوك...",اقيمت الدورة الاولمبية في طوكيو ٢٠٢١,"[اقيمت, الدورة, الاولمبية, في, طوكيو, ٢٠٢١]"


In [11]:
patterns = {
    'Person': r"\b(محمد|مريم|ياسمين)\b",  
    'Location': r"\b(دبي|باريس|اليابان|القاهرة)\b", 
    'Date': r"\b(\d{4}|العطلة الصيفية)\b",  
    'Organization': r"\b(جوجل|مايكروسوفت)\b"  
}

def rule_based_ner(sentence, patterns):
    entities = {}
    for entity_type, pattern in patterns.items():
        match = re.search(pattern, sentence)
        if match:
            entities[entity_type] = match.group(0)
    return entities

df['predicted_entities'] = df['sentence'].apply(lambda x: rule_based_ner(x, patterns))

df[['sentence', 'entities', 'predicted_entities']].head()


,sentence,entities,predicted_entities
0,زار محمد دبي في ٢٠٢٣.,"{'Person': 'محمد', 'Location': 'دبي', 'Date': ...","{'Person': 'محمد', 'Location': 'دبي', 'Date': ..."
1,حضرت مريم مؤتمر جوجل في باريس.,"{'Person': 'مريم', 'Organization': 'جوجل', 'Lo...","{'Person': 'مريم', 'Location': 'باريس', 'Organ..."
2,أسس أحمد شركة الفجر في عام ٢٠١٩.,"{'Person': 'أحمد', 'Organization': 'شركة الفجر...",{'Date': '٢٠١٩'}
3,تعمل سارة في مستشفى الجامعة في بيروت.,"{'Person': 'سارة', 'Organization': 'مستشفى الج...",{}
4,أقيمت الدورة الأولمبية في طوكيو ٢٠٢١.,"{'Event': 'الدورة الأولمبية', 'Location': 'طوك...",{'Date': '٢٠٢١'}


In [14]:
def prepare_ml_data(df):
    data = []
    for i, row in df.iterrows():
        tokens = row['tokens']
        entities = row['entities']
        token_labels = []
        
        for token in tokens:
            label = 'None'  
            for entity, value in entities.items():
                if token == value:
                    label = entity
            token_labels.append((token, label))
        
        data.append(token_labels)
    
    return data

ml_data = prepare_ml_data(df)




زار: None
محمد: Person
دبي: Location
في: None
٢٠٢٣: Date


In [15]:
def extract_features(tokens):
    return [{'word': token} for token in tokens]

def get_labels(token_labels):
    return [label for token, label in token_labels]

X_train = [extract_features([token for token, label in sentence]) for sentence in ml_data]
y_train = [get_labels(sentence) for sentence in ml_data]

crf = sklearn_crfsuite.CRF()
crf.fit(X_train, y_train)

y_pred = crf.predict(X_train)

print(metrics.flat_classification_report(y_train, y_pred))


              precision    recall  f1-score   support

        Date       1.00      0.40      0.57        15
       Event       0.00      0.00      0.00         6
    Location       1.00      0.71      0.83        21
        None       0.82      1.00      0.90       205
Organization       0.00      0.00      0.00         5
      Person       0.00      0.00      0.00        20

    accuracy                           0.83       272
   macro avg       0.47      0.35      0.38       272
weighted avg       0.75      0.83      0.77       272



C:\Users\moham\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\moham\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\moham\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

##### One thing to note, is that rule based used here is very limited to the data set at hand, which is a small one also, and yet, there is a lot of challenges. Rule based in arabic is tough, capturing dates maybe the easiest part, other than that, it is very challenging. In english, it is easier to get Nouns, especially names and so on and countries, using capitalization as a reference for example, in addition to some available libraries in text processing, which for example offers months names, country names, and so on and so forth.
##### Machine learning approach seems earier and more direct, training an ml model to do the job, especially after thorough data cleaning and tokenization